<a href="https://colab.research.google.com/github/Lucia-Garcia-Lado/Graph-Coloring-Models-for-Production-Line-Scheduling-Optimization/blob/main/graph_coloring_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph Coloring Models for Production Line Scheduling Optimization

> Lucía García Lado <br>
> Alejandro Sosa Corral

In [1]:
!pip install deap -q

In [2]:
from deap import base, creator, tools, algorithms
import random
import math

import numpy as np
import os

import matplotlib.pyplot as plt

In [3]:
# ================================
# Semilla global para reproducibilidad
# ================================
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)


In [4]:
# ------------------------------------------------------------
# 1. Carga de datos para dataset JSPLIB
# ------------------------------------------------------------

def load_instance(filename):
    operations = []

    with open(filename, 'r') as f:
        # Leemos las líneas, ignorando comentarios (#)
        lines = [line.strip() for line in f if line.strip() and not line.startswith("#")]

    # Eliminamos el header y tomamos el número de trabajos y máquinas
    header = lines[0].split()
    num_jobs = int(header[0])
    num_machines = int(header[1])

    # El resto de las líneas son trabajos
    job_lines = lines[1:]

    if len(job_lines) < num_jobs:
        raise ValueError(f"File format error: Expected {num_jobs} job lines, found {len(job_lines)}")

    for job_id in range(num_jobs):
        line = job_lines[job_id]
        values = list(map(int, line.split()))

        # Emparejamos: (Máquina, Duración)
        order = 0
        for i in range(0, len(values), 2):
            machine = values[i]
            duration = values[i+1]

            operations.append({
                "job": job_id,
                "machine": machine,
                "duration": duration,
                "order": order
            })
            order += 1

    return operations, num_jobs, num_machines

In [5]:
# ------------------------------------------------------------
# 2. Problema e hiperparámetros
# ------------------------------------------------------------

OPERATIONS, NUM_JOBS, NUM_MACHINES = load_instance("ft06")
NUM_OPS = len(OPERATIONS)

HORIZON = sum(op["duration"] for op in OPERATIONS)

print(f"Dataset loaded. Horizon set to: {HORIZON}")

# Variables genéticas
POPULATION_SIZE = max(NUM_OPS*15, 200)
NUM_GENERATIONS = max(NUM_OPS*5, 200)
CROSS_PROBABILITY = 0.8
MUT_PROBABILITY = 0.2

TOURNAMENT_SIZE = 2
ELITE_SIZE = 5

START_SHIFT = int(HORIZON*0.7)
END_SHIFT = 1

Dataset loaded. Horizon set to: 197


In [6]:
# 1. Mapa ID Máquina -> Lista de (índice, operación)
MACHINE_MAP = {}
machine_ids = set(op["machine"] for op in OPERATIONS)
for m in machine_ids:
    MACHINE_MAP[m] = [(i, op) for i, op in enumerate(OPERATIONS) if op["machine"] == m]

# 2. Mapa ID Trabajo -> Lista de (índice, operación) ordenadas por secuencia
JOB_MAP = {}
job_ids = set(op["job"] for op in OPERATIONS)
for j in job_ids:
    ops = [(i, op) for i, op in enumerate(OPERATIONS) if op["job"] == j]
    ops.sort(key=lambda x: x[1]["order"])
    JOB_MAP[j] = ops

# ------------------------------------------------------------
# 3. Función de Fitness (Evaluación)
# ------------------------------------------------------------

def evaluate(ind):
    # --- CONFLICTOS DE MÁQUINA ---
    machine_conflicts = 0

    for machine_id, machine_ops in MACHINE_MAP.items():

        # Comparamos cada par de operaciones en la misma máquina
        for i in range(len(machine_ops)):

          index_A, op_data_A = machine_ops[i]

          for j in range(i + 1, len(machine_ops)):

              index_B, op_data_B = machine_ops[j]

              start_A = ind[index_A]
              end_A = start_A + op_data_A["duration"]

              start_B = ind[index_B]
              end_B = start_B + op_data_B["duration"]

              # Comprobamos el overlap
              if start_A < end_B and start_B < end_A:
                  machine_conflicts += 1

    # --- CONFLICTOS DE PRECEDENCIA ---
    precedence_conflicts = 0

    for job_id, job_steps in JOB_MAP.items():

        # Iteramos sobre los pasos secuenciales
        for k in range(len(job_steps) - 1):
            current_step_idx, current_step_data = job_steps[k]
            next_step_idx, next_step_data = job_steps[k+1]

            current_step_end_time = ind[current_step_idx] + current_step_data["duration"]
            next_step_start_time = ind[next_step_idx]

            # El siguiente paso no puede empezar antes de que termine el actual
            if next_step_start_time < current_step_end_time:
                precedence_conflicts += 1

    # --- OBJETIVOS ---
    end_times = [ind[i] + OPERATIONS[i]["duration"] for i in range(NUM_OPS)]
    Cmax = max(end_times) if end_times else 0

    # Energía (Peak Load)
    events = []
    for i in range(NUM_OPS):
        start = ind[i]
        end = start + OPERATIONS[i]["duration"]
        events.append((start, 1))   # Máquina ON
        events.append((end, -1))    # Máquina OFF

    events.sort()

    peak_load = 0
    current_load = 0
    for _, change in events:
        current_load += change
        if current_load > peak_load:
            peak_load = current_load

    energy = peak_load

    # --- SUMA PONDERADA ---
    alpha = 1000  # Penalización alta por conflicto de máquina
    beta = 1000   # Penalización alta por conflicto de precedencia
    gamma = 10    # Peso para Carga Pico
    delta = 0.1   # Peso para Makespan

    total_cost = (alpha * machine_conflicts) + \
                 (beta * precedence_conflicts) + \
                 (gamma * energy) + \
                 (delta * Cmax)

    return total_cost,

In [7]:
# ------------------------------------------------------------
# 4. Operadores Genéticos
# ------------------------------------------------------------

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Generación de individuos factibles
def generate_feasible_sequence():
    ind = [0] * NUM_OPS

    start_range = int(HORIZON * 0.3)
    if start_range < 1: start_range = 1

    slack_range = int(HORIZON * 0.05)
    if slack_range < 1: slack_range = 1

    for job_id, steps in JOB_MAP.items():

        current_time = random.randint(0, start_range)

        for idx, op_data in steps:
            # Asignamos el tiempo actual a este paso
            ind[idx] = current_time

            duration = op_data["duration"]

            # Calculamos cuándo debe empezar el SIGUIENTE paso (Fin del actual + un "Slack" aleatorio (hueco)).
            slack = random.randint(0, slack_range)
            current_time += duration + slack

            if current_time > HORIZON:
                current_time = HORIZON

    return creator.Individual(ind)

# Generación de población
toolbox.register("individual", generate_feasible_sequence)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Operadores estándar
toolbox.register("evaluate", evaluate)
toolbox.register("select", tools.selTournament, tournsize=TOURNAMENT_SIZE)
toolbox.register("mate", tools.cxTwoPoint)

def multi_shift_mutation(ind, max_shift, indpb):  # Multi-shift nos permite explorar de manera más eficiente que el shift normal
    did_mutate = False
    for i in range(len(ind)):
        if random.random() < indpb:
            ind[i] += random.randint(-max_shift, max_shift)
            ind[i] = max(0, min(HORIZON, ind[i]))
            did_mutate = True

    if not did_mutate:  # Si no ha mutado ninguna en el bucle anterior mutamos una forzosamente
        idx = random.randrange(NUM_OPS)
        ind[idx] += random.randint(-max_shift, max_shift)
        # Mantener dentro de los límites válidos [0, HORIZON]
        ind[idx] = max(0, min(HORIZON, ind[idx]))
    return ind,

toolbox.register("mutate", multi_shift_mutation, max_shift=20, indpb=0.1)

In [8]:
def run_ga(seed):
    random.seed(seed)
    np.random.seed(seed)

    # Re-initialize population to ensure fresh start
    pop = toolbox.population(n=POPULATION_SIZE)

    # Evaluate initial population
    for ind in pop:
        ind.fitness.values = toolbox.evaluate(ind)

    # Evolution loop
    for gen in range(NUM_GENERATIONS):
        progress = gen / NUM_GENERATIONS
        current_max_shift = int(START_SHIFT * (1 - progress) + END_SHIFT * progress)

        # Update mutation operator
        toolbox.register("mutate", multi_shift_mutation, max_shift=current_max_shift, indpb=0.1)

        # Offspring and Evaluation
        offspring = algorithms.varAnd(pop, toolbox, CROSS_PROBABILITY, MUT_PROBABILITY)
        for ind in offspring:
            ind.fitness.values = toolbox.evaluate(ind)

        # Elitism & Replacement
        elite = tools.selBest(pop, ELITE_SIZE)
        rest = toolbox.select(offspring, POPULATION_SIZE - ELITE_SIZE)
        pop = elite + rest

    # Return best individual's fitness
    best_final = tools.selBest(pop, k=1)[0]
    return best_final.fitness.values[0]

def main():
    n_runs = 10
    results = []

    print(f"--- Starting {n_runs} runs for Single Objective GA (ft06 instance) ---")

    for i in range(1, n_runs + 1):
        fitness = run_ga(seed=i)
        results.append(fitness)
        print(f"Run {i}: Best Cost = {fitness:.2f}")

    # Calculate Statistics
    mean_score = np.mean(results)
    std_score = np.std(results)

    print("\n--- Statistical Results (Single Objective) ---")
    print(f"Runs: {n_runs}")
    print(f"Mean Cost: {mean_score:.2f}")
    print(f"Std Dev:   {std_score:.2f}")
    print(f"Best found: {np.min(results):.2f}")
    print(f"Worst found: {np.max(results):.2f}")

    return results # Return list for statistical testing later

if __name__ == "__main__":
    ga_results = main()

--- Starting 10 runs for Single Objective GA (ft06 instance) ---
Run 1: Best Cost = 41.40
Run 2: Best Cost = 40.50
Run 3: Best Cost = 41.60
Run 4: Best Cost = 41.20
Run 5: Best Cost = 41.50
Run 6: Best Cost = 41.70
Run 7: Best Cost = 42.30
Run 8: Best Cost = 41.30
Run 9: Best Cost = 41.80
Run 10: Best Cost = 50.10

--- Statistical Results (Single Objective) ---
Runs: 10
Mean Cost: 42.34
Std Dev:   2.62
Best found: 40.50
Worst found: 50.10


# Multiobejtivo

In [9]:
# ------------------------------------------------------------
# 2. Problema e hiperparámetros
# ------------------------------------------------------------

OPERATIONS, NUM_JOBS, NUM_MACHINES = load_instance("ft06")
NUM_OPS = len(OPERATIONS)

HORIZON = sum(op["duration"] for op in OPERATIONS)

print(f"Dataset loaded. Horizon set to: {HORIZON}")

# Variables genéticas
POPULATION_SIZE = max(NUM_OPS*15, 200)
NUM_GENERATIONS = max(NUM_OPS*5, 500)
CROSS_PROBABILITY = 0.8
MUT_PROBABILITY = 0.2

ELITE_SIZE = 5

START_SHIFT = int(HORIZON*1.2)
END_SHIFT = 1

Dataset loaded. Horizon set to: 197


In [10]:
# 1. Mapa ID Máquina -> Lista de (índice, operación)
MACHINE_MAP = {}
machine_ids = set(op["machine"] for op in OPERATIONS)
for m in machine_ids:
    MACHINE_MAP[m] = [(i, op) for i, op in enumerate(OPERATIONS) if op["machine"] == m]

# 2. Mapa ID Trabajo -> Lista de (índice, operación) ordenadas por secuencia
JOB_MAP = {}
job_ids = set(op["job"] for op in OPERATIONS)
for j in job_ids:
    ops = [(i, op) for i, op in enumerate(OPERATIONS) if op["job"] == j]
    ops.sort(key=lambda x: x[1]["order"])
    JOB_MAP[j] = ops

# ------------------------------------------------------------
# 3. Función de Fitness (Evaluación)
# ------------------------------------------------------------

def evaluate(ind):
    # --- CONFLICTOS DE MÁQUINA ---
    machine_conflicts = 0

    for machine_id, machine_ops in MACHINE_MAP.items():

        # Comparamos cada par de operaciones en la misma máquina
        for i in range(len(machine_ops)):
            for j in range(i + 1, len(machine_ops)):

                index_A, op_data_A = machine_ops[i]
                index_B, op_data_B = machine_ops[j]

                start_A = ind[index_A]
                end_A = start_A + op_data_A["duration"]

                start_B = ind[index_B]
                end_B = start_B + op_data_B["duration"]

                # Comprobamos el overlap
                if start_A < end_B and start_B < end_A:
                    machine_conflicts += 1

    # --- CONFLICTOS DE PRECEDENCIA ---
    precedence_conflicts = 0

    for job_id, job_steps in JOB_MAP.items():

        # Iteramos sobre los pasos secuenciales
        for k in range(len(job_steps) - 1):
            current_step_idx, current_step_data = job_steps[k]
            next_step_idx, next_step_data = job_steps[k+1]

            current_step_end_time = ind[current_step_idx] + current_step_data["duration"]
            next_step_start_time = ind[next_step_idx]

            # El siguiente paso no puede empezar antes de que termine el actual
            if next_step_start_time < current_step_end_time:
                precedence_conflicts += 1

    # --- OBJETIVOS ---
    end_times = [ind[i] + OPERATIONS[i]["duration"] for i in range(NUM_OPS)]
    Cmax = max(end_times) if end_times else 0

    # Energía (Peak Load)
    events = []
    for i in range(NUM_OPS):
        start = ind[i]
        end = start + OPERATIONS[i]["duration"]
        events.append((start, 1))   # Máquina ON
        events.append((end, -1))    # Máquina OFF

    events.sort()

    peak_load = 0
    current_load = 0
    for _, change in events:
        current_load += change
        if current_load > peak_load:
            peak_load = current_load

    energy = peak_load

    gamma = 10    # Peso para Carga Pico
    delta = 0.1

    return energy*gamma+Cmax*delta, (machine_conflicts+precedence_conflicts)*1000

In [11]:
# ------------------------------------------------------------
# 4. Operadores Genéticos
# ------------------------------------------------------------

creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -10.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)


toolbox = base.Toolbox()

# Generación de individuos factibles
def generate_feasible_sequence():
    ind = [0] * NUM_OPS

    start_range = int(HORIZON * 0.1)
    if start_range < 1: start_range = 1

    slack_range = int(HORIZON * 0.2)
    if slack_range < 1: slack_range = 1

    for job_id, steps in JOB_MAP.items():

        current_time = random.randint(0, start_range)

        for idx, op_data in steps:
            # Asignamos el tiempo actual a este paso
            ind[idx] = current_time

            duration = op_data["duration"]

            # Calculamos cuándo debe empezar el SIGUIENTE paso (Fin del actual + un "Slack" aleatorio (hueco)).
            slack = random.randint(0, slack_range)
            current_time += duration + slack

            if current_time > HORIZON:
                current_time = HORIZON

    return creator.Individual(ind)

# Generación de población
toolbox.register("individual", generate_feasible_sequence)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Operadores estándar
toolbox.register("evaluate", evaluate)
toolbox.register("select", tools.selNSGA2)
toolbox.register("mate", tools.cxTwoPoint)

def multi_shift_mutation(ind, max_shift, indpb):  # Multi-shift nos permite explorar de manera más eficiente que el shift normal
    did_mutate = False
    for i in range(len(ind)):
        if random.random() < indpb:
            ind[i] += random.randint(-max_shift, max_shift)
            ind[i] = max(0, min(HORIZON, ind[i]))
            did_mutate = True

    if not did_mutate:
        idx = random.randrange(NUM_OPS)
        ind[idx] += random.randint(-max_shift, max_shift)
        # Mantener dentro de los límites válidos [0, HORIZON]
        ind[idx] = max(0, min(HORIZON, ind[idx]))
    return ind,

toolbox.register("mutate", multi_shift_mutation, max_shift=20, indpb=0.1)

/usr/local/lib/python3.12/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [12]:
def run_nsga2(seed):
    random.seed(seed)
    np.random.seed(seed)

    pop = toolbox.population(n=POPULATION_SIZE)

    # Evaluate and initial sort
    for ind in pop:
        ind.fitness.values = toolbox.evaluate(ind)
    pop = toolbox.select(pop, len(pop))

    for gen in range(NUM_GENERATIONS):
        progress = gen / NUM_GENERATIONS
        current_max_shift = int(START_SHIFT * (1 - progress) + END_SHIFT * progress)
        toolbox.register("mutate", multi_shift_mutation, max_shift=current_max_shift, indpb=0.1)

        offspring = algorithms.varAnd(pop, toolbox, CROSS_PROBABILITY, MUT_PROBABILITY)
        for ind in offspring:
            ind.fitness.values = toolbox.evaluate(ind)

        # NSGA-II Selection
        elite = tools.selNSGA2(pop, 5) # Elite size 5
        combined = elite + offspring
        pop = tools.selNSGA2(combined, POPULATION_SIZE)

    # Sort by secondary objective (Penalty) first, then primary
    pop.sort(key=lambda ind: (ind.fitness.values[1], ind.fitness.values[0]))
    best_final = pop[0]

    # Return tuple: (Primary Objective, Penalty)
    return best_final.fitness.values

def main():
    n_runs = 5
    primary_objs = []
    penalties = []

    print(f"--- Starting {n_runs} runs for Multi-Objective GA (ft06 instance) ---")

    for i in range(1, n_runs + 1):
        # We assume result is (Energy+Makespan, Penalty)
        res = run_nsga2(seed=i)
        primary_objs.append(res[0])
        penalties.append(res[1])
        print(f"Run {i}: Obj={res[0]:.2f}, Penalty={res[1]:.2f}")

    print("\n--- Statistical Results (Multi-Objective) ---")
    print(f"Primary Objective -> Mean: {np.mean(primary_objs):.2f}, Std: {np.std(primary_objs):.2f}")
    print(f"Penalties         -> Mean: {np.mean(penalties):.2f},    Std: {np.std(penalties):.2f}")

    return primary_objs # Return for comparison

if __name__ == "__main__":
    nsga_results = main()

--- Starting 5 runs for Multi-Objective GA (ft06 instance) ---
Run 1: Obj=36.30, Penalty=0.00
Run 2: Obj=35.80, Penalty=0.00
Run 3: Obj=36.00, Penalty=0.00
Run 4: Obj=36.00, Penalty=0.00
Run 5: Obj=37.50, Penalty=0.00

--- Statistical Results (Multi-Objective) ---
Primary Objective -> Mean: 36.32, Std: 0.61
Penalties         -> Mean: 0.00,    Std: 0.00


In [13]:
# ------------------------------------------------------------
# 2. Carga del problema
# ------------------------------------------------------------

OPERATIONS, NUM_JOBS, NUM_MACHINES = load_instance("ft06")
NUM_OPS = len(OPERATIONS)

HORIZON = sum(op["duration"] for op in OPERATIONS)

print(f"Dataset loaded. Horizon set to: {HORIZON}")

# 1. Mapa ID Máquina -> Lista de (índice, operación)
MACHINE_MAP = {}
machine_ids = set(op["machine"] for op in OPERATIONS)
for m in machine_ids:
    MACHINE_MAP[m] = [(i, op) for i, op in enumerate(OPERATIONS) if op["machine"] == m]

# 2. Mapa ID Trabajo -> Lista de (índice, operación) ordenadas por secuencia
JOB_MAP = {}
job_ids = set(op["job"] for op in OPERATIONS)
for j in job_ids:
    ops = [(i, op) for i, op in enumerate(OPERATIONS) if op["job"] == j]
    ops.sort(key=lambda x: x[1]["order"])
    JOB_MAP[j] = ops

# ============================================================
#  Evaluación (coste escalar) + función auxiliar de componentes
# ============================================================

def evaluate_components(ind):
    # --- CONFLICTOS DE MÁQUINA ---
    machine_conflicts = 0

    for machine_id, machine_ops in MACHINE_MAP.items():

        # Comparamos cada par de operaciones en la misma máquina
        for i in range(len(machine_ops)):
            for j in range(i + 1, len(machine_ops)):

                index_A, op_data_A = machine_ops[i]
                index_B, op_data_B = machine_ops[j]

                start_A = ind[index_A]
                end_A = start_A + op_data_A["duration"]

                start_B = ind[index_B]
                end_B = start_B + op_data_B["duration"]

                # Comprobamos el overlap
                if start_A < end_B and start_B < end_A:
                    machine_conflicts += 1

    # --- CONFLICTOS DE PRECEDENCIA ---
    precedence_conflicts = 0

    for job_id, job_steps in JOB_MAP.items():

        # Iteramos sobre los pasos secuenciales
        for k in range(len(job_steps) - 1):
            current_step_idx, current_step_data = job_steps[k]
            next_step_idx, next_step_data = job_steps[k+1]

            current_step_end_time = ind[current_step_idx] + current_step_data["duration"]
            next_step_start_time = ind[next_step_idx]

            # El siguiente paso no puede empezar antes de que termine el actual
            if next_step_start_time < current_step_end_time:
                precedence_conflicts += 1

    # --- OBJETIVOS ---
    end_times = [ind[i] + OPERATIONS[i]["duration"] for i in range(NUM_OPS)]
    Cmax = max(end_times) if end_times else 0

    # Energía (Peak Load)
    events = []
    for i in range(NUM_OPS):
        start = ind[i]
        end = start + OPERATIONS[i]["duration"]
        events.append((start, 1))   # Máquina ON
        events.append((end, -1))    # Máquina OFF

    events.sort()

    peak_load = 0
    current_load = 0
    for _, change in events:
        current_load += change
        if current_load > peak_load:
            peak_load = current_load

    energy = peak_load

    return machine_conflicts, precedence_conflicts, Cmax, energy


def evaluate(ind):
    machine_conflicts, precedence_conflicts, Cmax, energy = evaluate_components(ind)

    # --- SUMA PONDERADA ---
    alpha = 1000  # Penalización alta por conflicto de máquina
    beta  = 1000  # Penalización alta por conflicto de precedencia
    gamma = 10    # Peso para Carga Pico
    delta = 0.1   # Peso para Makespan

    total_cost = (alpha * machine_conflicts) + \
                 (beta  * precedence_conflicts) + \
                 (gamma * energy) + \
                 (delta * Cmax)

    return total_cost


# ============================================================
#  Movimiento (vecino)
# ============================================================

def move(ind, max_shift=20):
    new = ind[:]  # copia superficial suficiente (lista de ints)

    # Número de operaciones a alterar (al menos 1)
    num_changes = random.randint(1, max(1, len(new) // 10))

    for _ in range(num_changes):
        idx = random.randrange(len(new))
        shift = random.randint(-max_shift, max_shift)
        new[idx] = max(0, min(HORIZON, new[idx] + shift))

    return new


# ============================================================
#  Simulated Annealing
# ============================================================

def simulated_annealing(
    max_iters=50000,
    T_init=1000.0,
    T_min=1e-3,
    alpha=0.995,
    max_shift=20,
    verbose=True
):
    """
    Algoritmo de Recocido Simulado (Simulated Annealing) para el job-shop.

    - max_iters: número máximo de iteraciones.
    - T_init: temperatura inicial.
    - T_min: temperatura mínima (criterio de parada).
    - alpha: factor de enfriamiento (T = alpha * T).
    - max_shift: desplazamiento máximo para generar vecinos.
    """

    # ===== 1. Solución inicial (factible-ish) usando tu generador =====
    current = generate_feasible_sequence()
    current_cost = evaluate(current)

    best = current[:]
    best_cost = current_cost

    T = T_init

    costs_history = []
    best_history = []

    for it in range(max_iters):
        if T < T_min:
            if verbose:
                print(f"Temperatura mínima alcanzada en iter {it}.")
            break

        # ===== 2. Generar vecino =====
        candidate = move(current, max_shift=max_shift)
        candidate_cost = evaluate(candidate)

        # ===== 3. Criterio de aceptación =====
        delta = candidate_cost - current_cost

        if delta < 0:
            # Mejor solución -> aceptamos siempre
            accept = True
        else:
            # Peor solución -> aceptamos con probabilidad exp(-delta / T)
            prob = math.exp(-delta / T) if T > 0 else 0
            accept = (random.random() < prob)

        if accept:
            current = candidate
            current_cost = candidate_cost

            # Actualizar mejor global
            if current_cost < best_cost:
                best = current[:]
                best_cost = current_cost

        # ===== 4. Enfriamiento =====
        T *= alpha

        # ===== 5. Guardar histórico para graficar =====
        costs_history.append(current_cost)
        best_history.append(best_cost)

        if verbose and it % 1000 == 0:
            print(f"Iter {it:6d} | T={T:8.4f} | coste actual={current_cost:.2f} | mejor={best_cost:.2f}")

    # ===== 6. Resultados finales =====
    mc, pc, Cmax, energy = evaluate_components(best)
    alpha_p = 1000
    beta_p  = 1000
    penalty = alpha_p * mc + beta_p * pc  # parte de penalización pura

    print("\n--- Resultado Simulated Annealing ---")
    print(f"Mejor coste escalar: {best_cost:.2f}")
    print(f"Conflictos máquina: {mc}")
    print(f"Conflictos precedencia: {pc}")
    print(f"Cmax: {Cmax}")
    print(f"Energía (peak load): {energy}")
    print(f"Penalización pura (alpha*mc + beta*pc): {penalty}")
    print("Solución FACTIBLE" if penalty == 0 else f"Solución INFACTIBLE (penalty={penalty})")

    return best, best_cost


def main():
    n_runs = 10
    results = []

    print(f"--- Starting {n_runs} runs for Simulated Annealing ---")

    for i in range(1, n_runs + 1):
        # Set seed inside the loop before calling the function
        random.seed(i)
        np.random.seed(i)

        # Run SA
        best_solution, best_cost = simulated_annealing(
            max_iters=10000000,
            T_init=2500.0,
            T_min=1e-3,
            alpha=0.9999,
            max_shift=3,
            verbose=False # Turn off verbose to keep console clean
        )

        results.append(best_cost)
        print(f"Run {i}: Best Cost = {best_cost:.2f}")

    # Statistics
    mean_score = np.mean(results)
    std_score = np.std(results)

    print("\n--- Statistical Results (Simulated Annealing) (ft06 instance) ---")
    print(f"Runs: {n_runs}")
    print(f"Mean Cost: {mean_score:.2f}")
    print(f"Std Dev:   {std_score:.2f}")

    return results

if __name__ == "__main__":
    sa_results = main()


Dataset loaded. Horizon set to: 197
--- Starting 10 runs for Simulated Annealing ---

--- Resultado Simulated Annealing ---
Mejor coste escalar: 4032.70
Conflictos máquina: 0
Conflictos precedencia: 4
Cmax: 127
Energía (peak load): 2
Penalización pura (alpha*mc + beta*pc): 4000
Solución INFACTIBLE (penalty=4000)
Run 1: Best Cost = 4032.70

--- Resultado Simulated Annealing ---
Mejor coste escalar: 3059.30
Conflictos máquina: 2
Conflictos precedencia: 1
Cmax: 193
Energía (peak load): 4
Penalización pura (alpha*mc + beta*pc): 3000
Solución INFACTIBLE (penalty=3000)
Run 2: Best Cost = 3059.30

--- Resultado Simulated Annealing ---
Mejor coste escalar: 3032.80
Conflictos máquina: 0
Conflictos precedencia: 3
Cmax: 128
Energía (peak load): 2
Penalización pura (alpha*mc + beta*pc): 3000
Solución INFACTIBLE (penalty=3000)
Run 3: Best Cost = 3032.80

--- Resultado Simulated Annealing ---
Mejor coste escalar: 4031.00
Conflictos máquina: 0
Conflictos precedencia: 4
Cmax: 110
Energía (peak load): 